### google translator自动文档翻译器

目前Google Translator中英互译，有5000单字上限。
可大批量，连续翻译长文件，打破Google Translator的5000单字上限。
支持全中->全英，或全英->全中。

##### 条件
- python 3.6
- 环境中有Selenium
- 自己的浏览器能够登录Google Translator
- 使用Headless Chrome来渲染网页，禁止GPU渲染

##### 雏形
Input一个Word或Excel文件，Output一个新的Word或Excel文件。
后续会使用一个OCR的转换器，将pdf中的中文字符转化为.txt，再扔进里面去转换。

In [12]:
import re
import docx2txt
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import docx
import time 

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')

In [19]:
def input_(file):
    if file[7:] == ".docx":
        text = docx2txt.process(file)
        return text
    else:
        table = pd.read_excel("source3.xlsx")
        text_l = table[table.columns[0]][0:30]
        text = "￥-￥".join(text_l)
        return text_l,text
        
def translate_process(translator_source):
    pattern_cn = re.compile(r"[\u4e00-\u9fa5].*?") #匹配中文 
    if re.search(pattern_cn,translator_source): #判断是英译中还是中译英
        if len(translator_source) < 3500:
            text = words_below_5000(translator_source)
            return text
        else:
            text = words_over_5000(translator_source)
            return text
    else:
        if len(translator_source) < 3500:
            text = translate_model_en(translator_source)
            return text
        else:
            text = words_over_5000(translator_source)
            return text

def translate_model_cn(translator_source):
    driver = webdriver.Chrome()
    driver.get("https://translate.google.com/")
    driver.find_element_by_xpath('//*[@id="gt-sl-sugg"]/div/div[5]').click()
    driver.find_element_by_css_selector('div#gt-lang-right > div#gt-lang-tgt > div#gt-tl-sugg > div.sl-sugg-button-container >div#sugg-item-en').click()
    input_area = driver.find_element_by_xpath('//*[@id="source"]')
    input_area.send_keys(translator_source)
    driver.find_element_by_xpath('//*[@id="gt-submit"]').click()
    driver.refresh()
    time.sleep(10)
    html_soup = BeautifulSoup(driver.page_source,"lxml")
    html_text = html_soup.find("span",{"id":"result_box"})
    text = html_text.get_text()
    driver.close()
    return text

def translate_model_en(translator_source):
    driver = webdriver.Chrome()
    driver.get("https://translate.google.cn/")
    input_area = driver.find_element_by_xpath('//*[@id="source"]')
    input_area.send_keys(translator_source)
    time.sleep(5)
    driver.find_element_by_xpath('//*[@id="gt-submit"]').click()
    time.sleep(5)
    driver.refresh()
    html_soup = BeautifulSoup(driver.page_source,"lxml")
    html_text = html_soup.find("span",{"id":"result_box"})
    text = html_text.get_text()
    driver.close()
    return text

def words_below_5000(translator_source):
    text = translate_model_en(translator_source)
    return text

def words_over_5000(translator_source):
    list_temp_save = []
    total_turn = (int(len(translator_source)/3500)) + 1
    for turn in range(total_turn):
        text = translate_model_en(translator_source[int(3500*turn):int((3500 * (turn + 1)))])
        list_temp_save.append(text)
        time.sleep(5)
    text = "".join(list_temp_save)
    return text

def output_(text_l,translated_text):
    pattern = re.compile("¥ - ¥")
    if re.search(pattern,translated_text):
        l_t = translated_text.split("¥ - ¥")
        output_d = pd.DataFrame([list(text_l),l_t]).T
        print(output_d)
        output_d.to_excel("output.xlsx")
    else:
        print(translated_text)

def main(file):
    text_l,text = input_(file)
    translated_text = translate_process(text)
    output_(text_l,translated_text)

if __name__ == '__main__':
    main("source3.xlsx")

                                                    0  \
0   Telephones for cellular networks or for other ...   
1   Automatic data processing machines; portable, ...   
2   Communication apparatus (excluding telephone s...   
3         Commodities not specified according to kind   
4   Machinery; parts and accessories (other than c...   
5   Tricycles, scooters, pedal cars and similar wh...   
6   Soya beans; other than seed, whether or not br...   
7         Commodities not specified according to kind   
8   Vehicles; with only spark-ignition internal co...   
9                        Electrical static converters   
10  Monitors; other than cathode-ray tube; capable...   
11  Games; video game consoles and machines, other...   
12  Units of automatic data processing machines; p...   
13  Semiconductor media; solid-state non-volatile ...   
14  Oils; petroleum oils and oils obtained from bi...   
15  Seats; with wooden frames, upholstered, (exclu...   
16  Reception apparatus for tel